## Import necessary libraries
This cell imports the required libraries for the code companion, including NumPy, Pandas, scikit-learn's cosine similarity, and the Surprise library for matrix factorization.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

## Load and preprocess the MovieLens 100K dataset
In this cell, we load the MovieLens 100K dataset files (u.item and u.data) into Pandas DataFrames. We merge the movies and ratings DataFrames based on the movie_id column.

In [2]:
# Load the MovieLens 100K dataset
movies = pd.read_csv('../data/ml-100k/u.item', sep='|', encoding='latin-1', header=None, usecols=[0, 1],
                     names=['movie_id', 'title'])
ratings = pd.read_csv('../data/ml-100k/u.data', sep='\t', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Merge movies and ratings dataframes
df = pd.merge(movies, ratings, on='movie_id')

df.head()

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


## User-Based Collaborative Filtering

This cell defines the user_based_cf function that implements User-Based Collaborative Filtering. It finds similar users based on their rating patterns using cosine similarity and recommends movies that similar users have rated highly.

In [3]:
# User-Based Collaborative Filtering
def user_based_cf(user_id, n_recommendations):
    user_item_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')
    user_similarity = cosine_similarity(user_item_matrix.fillna(0))
    
    user_index = user_id - 1
    similar_users = user_similarity[user_index].argsort()[::-1][1:]
    
    recommendations = []
    for user in similar_users:
        user_recommendations = user_item_matrix.iloc[user].dropna().sort_values(ascending=False)
        recommendations.extend(user_recommendations.index)
        if len(recommendations) >= n_recommendations:
            break
    
    recommended_movies = movies[movies['movie_id'].isin(recommendations[:n_recommendations])]
    return recommended_movies

## Item-Based Collaborative Filtering
The item_based_cf function in this cell implements Item-Based Collaborative Filtering. It finds similar movies based on user ratings using cosine similarity and recommends movies that are similar to the given movie.

In [4]:
# Item-Based Collaborative Filtering
def item_based_cf(movie_id, n_recommendations):
    movie_user_matrix = df.pivot_table(index='movie_id', columns='user_id', values='rating')
    item_similarity = cosine_similarity(movie_user_matrix.fillna(0))
    
    movie_index = movie_id - 1
    similar_movies = item_similarity[movie_index].argsort()[::-1][1:]
    
    recommended_movies = movies[movies['movie_id'].isin(similar_movies[:n_recommendations])]
    return recommended_movies

## Matrix Factorization (SVD)
This cell defines the matrix_factorization function that uses the Singular Value Decomposition (SVD) algorithm from the Surprise library to factorize the user-item matrix and make recommendations based on the predicted ratings.

In [5]:
# Matrix Factorization (SVD)
def matrix_factorization(user_id, n_recommendations):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2)
    
    algo = SVD()
    algo.fit(trainset)
    
    user_ratings = df[df['user_id'] == user_id][['movie_id', 'rating']]
    user_ratings['pred_rating'] = user_ratings['movie_id'].apply(lambda x: algo.predict(user_id, x).est)
    user_ratings = user_ratings.sort_values(by='pred_rating', ascending=False)
    
    recommended_movies = movies[movies['movie_id'].isin(user_ratings['movie_id'][:n_recommendations])]
    return recommended_movies

## Example usage
In this cell, we provide example usage of the three recommender system functions. We specify a user_id, movie_id, and the desired number of recommendations (n_recommendations). We call each recommender system function and prints the recommended movies for the given user or movie.

In [6]:
# Example usage
user_id = 1
movie_id = 1
n_recommendations = 5

print("User-Based Collaborative Filtering:")
user_based_cf(user_id, n_recommendations).head()

User-Based Collaborative Filtering:


,movie_id,title
51,52,"Madness of King George, The (1994)"
55,56,Pulp Fiction (1994)
57,58,Quiz Show (1994)
88,89,Blade Runner (1982)
173,174,Raiders of the Lost Ark (1981)


In [7]:
print("\nItem-Based Collaborative Filtering:")
item_based_cf(movie_id, n_recommendations).head()


Item-Based Collaborative Filtering:


,movie_id,title
48,49,I.Q. (1994)
115,116,Cold Comfort Farm (1995)
119,120,Striptease (1996)
179,180,Apocalypse Now (1979)
403,404,Pinocchio (1940)


In [8]:
print("\nMatrix Factorization (SVD):")
matrix_factorization(user_id, n_recommendations).head()


Matrix Factorization (SVD):


,movie_id,title
49,50,Star Wars (1977)
99,100,Fargo (1996)
113,114,Wallace & Gromit: The Best of Aardman Animatio...
126,127,"Godfather, The (1972)"
168,169,"Wrong Trousers, The (1993)"
